## Load Dataset

In [1]:
!pip install datasets accelerate transformers[torch]


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install transformers --upgrade


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset, DatasetDict
diary_dataset_train = load_dataset("csv", data_files = "diary_train.csv")
diary_dataset_test = load_dataset("csv", data_files = "diary_test.csv")

In [3]:
# hashtag열 제거
diary_dataset_train = diary_dataset_train['train'].remove_columns('hashtag')
diary_dataset_test = diary_dataset_test['train'].remove_columns('hashtag')

In [4]:
diary_dataset = DatasetDict({
    "train": diary_dataset_train,
    "test": diary_dataset_test
})

In [5]:
diary_dataset

DatasetDict({
    train: Dataset({
        features: ['diary', 'summary'],
        num_rows: 2703
    })
    test: Dataset({
        features: ['diary', 'summary'],
        num_rows: 301
    })
})

In [6]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
device

device(type='cuda')

## Summarization

In [8]:
from transformers import (
    AutoTokenizer,
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from tokenizers import Tokenizer
from typing import Dict, List, Optional
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from typing import Dict

In [9]:
# Load Model and Tokenize
model_name = "gogamza/kobart-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [10]:
max_input_length = 512
max_target_length = 150

def preprocess_function(ex):
  model_inputs = tokenizer(ex['diary'], max_length = max_input_length, padding = 'max_length', truncation = True)

  # 타겟을 위한 토크나이저 설정
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(ex['summary'], max_length = max_target_length, padding = 'max_length', truncation = True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [11]:
tokenized_dataset = diary_dataset.map(preprocess_function)

Map:   0%|          | 0/2703 [00:00<?, ? examples/s]

C:\Users\user\Documents\EWHA\log\log\web\flask-server\venv\Lib\site-packages\transformers\tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/301 [00:00<?, ? examples/s]

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['diary', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2703
    })
    test: Dataset({
        features: ['diary', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 301
    })
})

In [13]:
tokenized_dataset['train'].to_pandas().to_csv("tokenized_dataset_train_summary.csv", index = False)
tokenized_dataset['test'].to_pandas().to_csv("tokenized_dataset_test_summary.csv", index = False)

In [14]:
tokenized_dataset = tokenized_dataset.remove_columns(diary_dataset['train'].column_names)

In [15]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2703
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 301
    })
})

In [16]:
# Train data max input tokens
max(len(x) for x in tokenized_dataset['train']['input_ids'])

512

In [17]:
# Test data max input tokens
max(len(x) for x in tokenized_dataset['test']['input_ids'])

512

In [18]:
# Train data max output tokens
max(len(x) for x in tokenized_dataset['train']['labels'])

150

In [19]:
# Test data max output tokens
max(len(x) for x in tokenized_dataset['test']['labels'])

150

In [20]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

## Metrics : Rouge score

In [21]:
from datasets import load_metric
rouge_score = load_metric('rouge')

C:\Users\user\AppData\Local\Temp\ipykernel_6292\844052376.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_score = load_metric('rouge')
C:\Users\user\Documents\EWHA\log\log\web\flask-server\venv\Lib\site-packages\datasets\load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
import evaluate 
rouge_score = evaluate.load('rouge')

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    rouge_output = rouge_score.compute(predictions=decoded_preds, references=decoded_labels)
    return rouge_output

## Train

In [33]:
!pip install ipywidgets


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
batch_size = 8
epochs = 50
logging_steps = len(tokenized_dataset['train']) // batch_size
model_path = "C:/Users/user/Documents/EWHA/log/log/modelling/summary2/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path,
    overwrite_output_dir=True,
    evaluation_strategy = 'steps',
    learning_rate = 5.6e-5,
    weight_decay = 0.001,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=1000,
    warmup_steps=300,
    prediction_loss_only=True,
    predict_with_generate = True,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub = True
    )

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    callbacks=[early_stopping_callback],
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

In [27]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.483200,0.392476
1000,0.249400,0.388146
1500,0.124000,0.444324
2000,0.071300,0.467032
2500,0.037600,0.496729


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2500, training_loss=0.3931010063171387, metrics={'train_runtime': 886.7759, 'train_samples_per_second': 152.406, 'train_steps_per_second': 19.058, 'total_flos': 6095230504796160.0, 'train_loss': 0.3931010063171387, 'epoch': 7.4})

In [ ]:
evaluation_results = trainer.evaluate()

In [30]:
evaluation_results
# eval loss : 0.4011 (weight_decay: 0.01)

{'eval_loss': 0.38814619183540344,
 'eval_runtime': 3.8091,
 'eval_samples_per_second': 79.022,
 'eval_steps_per_second': 9.976,
 'epoch': 7.4}

In [31]:
trainer.save_model('./summary2')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


In [32]:
trainer.push_to_hub(commit_message = "Summarize a diary", tags = 'kobart-summarization-diary')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


CommitInfo(commit_url='https://huggingface.co/jjae/summary2/commit/5256d557edf86b9bb14fe530a9da14d528885b61', commit_message='Summarize a diary', commit_description='', oid='5256d557edf86b9bb14fe530a9da14d528885b61', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
tokenizer.push_to_hub(repo_id = 'summary2')

README.md:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

C:\Users\user\Documents\EWHA\log\log\web\flask-server\venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--jjae--summary2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/jjae/summary2/commit/9e2c4932adbd5af73c8aa27dea3e60694c4b4239', commit_message='Upload tokenizer', commit_description='', oid='9e2c4932adbd5af73c8aa27dea3e60694c4b4239', pr_url=None, pr_revision=None, pr_num=None)

In [82]:
def diary_summary(text):
  # Encode input text
  input_ids = tokenizer.encode(text, return_tensors = 'pt').to(device)
  # Generate summary text ids
  summary_text_ids = model.generate(input_ids = input_ids,
                                    bos_token_id = model.config.bos_token_id,
                                    eos_token_id = model.config.eos_token_id,
                                    length_penalty = 2.0,
                                    max_length = 150,
                                    num_beams = 2)
  return tokenizer.decode(summary_text_ids[0], skip_special_tokens = True)

In [43]:
def diary_summary(text, label):
  # Encode input text
  input_ids = tokenizer.encode(text, return_tensors = 'pt').to(device)
  # Generate summary text ids
  summary_text_ids = model.generate(input_ids = input_ids,
                                    bos_token_id = model.config.bos_token_id,
                                    eos_token_id = model.config.eos_token_id,
                                    length_penalty = 2.0,
                                    max_length = 150,
                                    num_beams = 2)
  print("원문:", text, '\n')
  print("label:", label, '\n')
  print("요약본:",tokenizer.decode(summary_text_ids[0], skip_special_tokens = True))

In [44]:
diary_summary(diary_dataset['test']['diary'][0], diary_dataset['test']['summary'][0])

원문: 나는 건강을 유지하기 위해 꾸준한 운동과 올바른 식습관을 실천하고 있으며, 이를 위한 자기 동기부여는 나에게 큰 의미를 가지고 있어요. 자기 동기부여는 건강한 라이프스타일을 유지하는 데 있어서 가장 중요한 요소 중 하나에요. 처음에는 건강을 유지하기 위해 노력하는 것이 힘들고 어려웠어요. 하지만 나 자신에게 건강한 몸과 마음을 유지하는 것이 얼마나 중요한지 깨달았고, 그것이 나에게 행복과 만족을 주는 것을 깨달았어요. 자기 동기부여를 유지하기 위해 나는 목표를 설정하고 그것을 달성하기 위해 노력하고 있어요. 목표는 건강하고 활발한 삶을 살아가는 것이에요. 그것을 위해 꾸준한 운동과 올바른 식습관을 실천하고, 자기 관리에도 신경을 쓰고 있어요. 목표를 향해 나아가는 과정에서의 자기 성취와 성장은 나에게 큰 자신감과 만족감을 주어요. 또한, 주변의 영향력도 나에게 자기 동기부여를 줄 수 있는 중요한 요소 중 하나에요. 가족이나 친구들과 함께 운동을 하거나 건강한 식사를 즐길 때, 서로를 격려하고 지지해주며 동기부여를 공유할 수 있어요. 이러한 동료들과의 관계는 나에게 힘과 용기를 주고, 더욱 열심히 노력하고 발전하는 원동력이 되어줘요. 자기 동기부여는 건강한 라이프스타일을 유지하는 데 있어서 필수적인 요소라고 생각해요. 나는 건강한 몸과 마음을 갖고 행복하고 풍요로운 삶을 살아가기 위해 자기 동기부여를 지속적으로 유지하고 발전시키고 있어요. 

label: 건강한 라이프스타일을 유지하는 데 있어서 자기 동기부여는 매우 중요한 요소에요. 목표를 설정하고 그것을 달성하기 위해 노력하며, 자기 성취와 성장을 경험함으로써 자신감과 만족감을 얻을 수 있어요. 주변의 영향력과 동료들과의 관계도 자기 동기부여를 공유하고 힘을 얻는 데 도움을 줘요. 

요약본: 건강한 라이프스타일을 유지하는 데 필요한 자기 동기부여에 대해 생각하게 되었습니다. 자기 동기부여는 건강한 몸과 마음을 유지하는 데 가장 중요한 요소이며, 꾸준한 운동과 올바른 식습관을 실천하고 자기 관리에도 신